In [27]:
#import neccessary modules
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [26]:
# helper functions
from timeseries_helpers import datasetstorer
from timeseries_helpers import dataset_importer
from timeseries_helpers import database_importer
from timeseries_helpers import standardizer
from data_preprocessing import extract_accelerometer_data, csv_to_dataset_list
from timeseries_helpers import classificationHelpers

In [3]:
# Read the CSV files
data_1 = pd.read_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gerade.csv')
data_2 = pd.read_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Kinnhacken.csv')
data_3 = pd.read_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Kopfhacken.csv')
data_4 = pd.read_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gemischt_02.csv')
data_5 = pd.read_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gemischt_01.csv')

# Concatenate the dataframes
merged_data = pd.concat([data_1, data_2, data_3, data_4, data_5])

# Write the merged dataframe to a new CSV file
merged_data.to_csv('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.csv', index=False)

ds = csv_to_dataset_list('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 4 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   accelerometerTimestamp_sinceReboot(s)  89378 non-null  float64
 1   accelerometerAccelerationX(G)          89378 non-null  float64
 2   accelerometerAccelerationY(G)          89378 non-null  float64
 3   accelerometerAccelerationZ(G)          89378 non-null  float64
dtypes: float64(4)
memory usage: 2.7 MB
Data imported without any errors!
 ##### Information: ####
 Imported datasets: 1


In [39]:
data = extract_accelerometer_data('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gerade.csv')

def csv_to_json(file_path):
    dataset = []
    # Read the CSV file and morph it into a indexed Dictionary
    data = extract_accelerometer_data(file_path)
    
    # insert raws into dataset
    dataset = {'raws': data, 'label': 0, 'count': len(data)}
    
    for idx,e in data:
        if idx < 100:
            print(idx)
    

In [40]:
csv_to_json('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gerade.csv')

ValueError: too many values to unpack (expected 2)

In [5]:
## period length in ms
periodLengthMS = 1000

## sample rate in µs
sampleRateUS = 10000

## test/train ratio
trainDataRatio = 0.7
trainDataAbs = 100*trainDataRatio

## Json dataset path
jsonDSpath = '/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json'

In [11]:
#with open('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json', 'r') as f:
#    jsnDataset = json.load(f)

tweets = []
with open('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json', 'r') as file:
    for line in file:
        tweets.append(json.loads(line))

dl = database_importer.jsonData_to_dataset_in_timedifference_us(tweets)

KeyError: 'raws'

In [6]:
buff = classificationHelpers.prepareDataset(jsonDSpath, periodLengthMS, sampleRateUS)

JSONDecodeError: Extra data: line 2 column 1 (char 2268)

In [6]:
# converts the given list of dataframes to one dataframe
df_list = []
for idx,e in enumerate(ds):
    df = e.assign(punch_id=idx)
    df['label'] = 0
    df_list.append(df)
df_res = pd.concat(df_list)
df_new = df_res.reset_index(drop=True)
df_new = df_new.rename(index=str, columns={"accelerometerAccelerationX(G)": "a_x", "accelerometerAccelerationY(G)": "a_y", "accelerometerAccelerationZ(G)":"a_z"})

89378

In [8]:
df_new.head()

,accelerometerTimestamp_sinceReboot(s),a_x,a_y,a_z,punch_id,label
0,15981.514496,-0.059662,-0.098602,-0.960510,0,0
1,15981.524465,-0.021240,-0.115128,-0.966843,0,0
2,15981.534434,-0.059235,-0.121017,-0.965866,0,0
3,15981.544403,-0.051376,-0.108932,-0.978439,0,0
4,15981.554372,-0.038437,-0.100632,-0.986923,0,0


FileNotFoundError: [Errno 2] No such file or directory: '../../Datasets/complete_smartpunch_dataset_AsOneDataFrame_periodMS2000_sampleUS1000.csv'